In [ ]:
import os

from matplotlib.pyplot import imshow, figure, bar, hist
from skimage.measure import compare_ssim as ssim
from skimage import io
from skimage import exposure
from skimage.transform import resize

import imagehash
from PIL import Image
import numpy as np
from shutil import copyfile

In [ ]:
PATH_TO_TRAIN = "../../data/input/data_stage1_train.zip"

In [ ]:
INITIAL_SAMPLE = "01d44a26f6680c42ba94c9bc6339228579a95d0e2695b149b7cc0c9592b21baf"

file_name = os.path.join(PATH_TO_TRAIN,"{}/images/{}.png".format(INITIAL_SAMPLE,INITIAL_SAMPLE))
file_png = io.imread(file_name, as_grey=True) 
file_png = exposure.equalize_hist(file_png)
file_png = file_png * 255
file_png = file_png.astype('uint8')

img_to_test = Image.fromarray(file_png)
print(file_png.min(), file_png.max())


In [ ]:
%matplotlib inline
array1 = np.asarray(img_to_test)
imshow(array1)
figure()
array1_flattened = array1.flatten()
hist(array1_flattened, bins = 30)
figure()
print(array1.shape, array1.max())
array1_flattened = array1.flatten()
hist(array1_flattened, bins = 30)

In [ ]:
images_to_process = []

for current_directory in os.listdir(PATH_TO_TRAIN):
    current_image = {}
    current_image["id"] = current_directory
    current_image["images"] = []
    current_image["masks"] = []
    for current_image_file in os.listdir(os.path.join(PATH_TO_TRAIN,current_directory,"images")):
        current_image["images"].append(os.path.join(PATH_TO_TRAIN,current_directory,"images", current_image_file))
    
    for current_image_mask in os.listdir(os.path.join(PATH_TO_TRAIN,current_directory,"masks")):
        current_image["masks"].append(os.path.join(PATH_TO_TRAIN,current_directory,"masks", current_image_mask))
   
    images_to_process.append(current_image)

In [ ]:
for current_image in images_to_process:
    file_png = io.imread(current_image['images'][0], as_grey=True) 
    file_png = exposure.equalize_hist(file_png)
    file_png = file_png * 255
    file_png = file_png.astype('uint8')
    #img_to_test = Image.fromarray(file_png)
    #array1 = np.asarray(img_to_test)
    #figure()
    #imshow(array1)
    #hash1 = imagehash.phash(img_to_test, hash_size=5)
    #print(hash1)
    file_png = resize(file_png,array1.shape)
    file_png = exposure.equalize_hist(file_png)
    file_png = file_png * 255
    file_png = file_png.astype('uint8')
    
    #print(file_png.dtype, array1.dtype)
    #print(file_png.shape, array1.shape)
    similarity = ssim(array1, file_png)
    #print(similarity)
    current_image['hash'] = similarity



In [ ]:
hashes = [x['hash'] for x in images_to_process]
hashes

In [ ]:
hashes = sorted(hashes, reverse = True)
hashes = hashes[:10]
print(hashes)

ids = [str(x['hash'])[:5] + '/' + x['images'][0] for x in images_to_process if x['hash'] in hashes ]

for file in ids:
    correct_file = "/".join(file.split("/")[1:])
    final_file = '../../data/' + file.split("/")[0] + "." + file.split("/")[8]
    print(final_file)
    copyfile(correct_file,final_file)
